In [2]:
import pandas as pd

# Loading Dataset to Pandas DataFrame:

In [3]:
data_frame = pd.read_csv('C:/Users/sa/Desktop/csv_file.csv')
data_frame

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Franciso,Seattle
0,ALASKA,on time,497,221,212,503,1841
1,ALASKA,delayed,62,12,20,102,305
2,AM WEST,on time,694,4840,383,320,201
3,AM WEST,delayed,117,415,65,129,61


# Tidying Data:

First, I have names the columns that were missing a name

In [4]:
data_frame.rename(columns={'Unnamed: 0': 'Airline', 'Unnamed: 1': 'Arrival'}, inplace=True)
data_frame

,Airline,Arrival,Los Angeles,Phoenix,San Diego,San Franciso,Seattle
0,ALASKA,on time,497,221,212,503,1841
1,ALASKA,delayed,62,12,20,102,305
2,AM WEST,on time,694,4840,383,320,201
3,AM WEST,delayed,117,415,65,129,61


In [5]:
new = pd.melt(data_frame,["Airline",'Arrival'],var_name='City',value_name='Value')
new

,Airline,Arrival,City,Value
0,ALASKA,on time,Los Angeles,497
1,ALASKA,delayed,Los Angeles,62
2,AM WEST,on time,Los Angeles,694
3,AM WEST,delayed,Los Angeles,117
4,ALASKA,on time,Phoenix,221
5,ALASKA,delayed,Phoenix,12
6,AM WEST,on time,Phoenix,4840
7,AM WEST,delayed,Phoenix,415
8,ALASKA,on time,San Diego,212
9,ALASKA,delayed,San Diego,20


In [6]:
index = ["Airline",'City']
tidy_table = new.pivot_table(index=index,columns='Arrival',values='Value')
tidy_table.reset_index(drop=False, inplace=True)
tidy_table.columns.name = ''
tidy_table

,Airline,City,delayed,on time
0,ALASKA,Los Angeles,62,497
1,ALASKA,Phoenix,12,221
2,ALASKA,San Diego,20,212
3,ALASKA,San Franciso,102,503
4,ALASKA,Seattle,305,1841
5,AM WEST,Los Angeles,117,694
6,AM WEST,Phoenix,415,4840
7,AM WEST,San Diego,65,383
8,AM WEST,San Franciso,129,320
9,AM WEST,Seattle,61,201



After a few tidying steps the data is now tidy and clear. The data can be used to perform analysis and derive insights from. 

# Data Analysis:

## 1. First I will find the on time performance of both airlines for each city.

In [7]:
tidy_table_2 = tidy_table.pivot_table(index='Airline',columns='City',values='on time')
tidy_table_2.columns.name = ''
tidy_table_2

,Los Angeles,Phoenix,San Diego,San Franciso,Seattle
Airline,,,,,
ALASKA,497,221,212,503,1841
AM WEST,694,4840,383,320,201


This table shows the on time arrivals for all five cities.

In [8]:
tidy_table_2.loc[:,'Sum']= tidy_table_2.sum(axis=1)
tidy_table_2

,Los Angeles,Phoenix,San Diego,San Franciso,Seattle,Sum
Airline,,,,,,
ALASKA,497,221,212,503,1841,3274
AM WEST,694,4840,383,320,201,6438


Here, I added a new column named Sum, which is the sum of all on time arrivals for both airlines.

In [9]:
col_names = list(tidy_table_2)

for x in range(2):
    total = tidy_table_2.iloc[x]['Sum']

    for i in col_names:
        value = tidy_table_2.iloc[x][i]
        #if value == total:
        #    break
        value = value/total
        percentage = value*100
        percentage = round(percentage, 1)
        #print(percentage)
        if x == 0:
            tidy_table_2.loc['ALASKA', i] = percentage
        else:
            tidy_table_2.loc['AM WEST', i] = percentage
tidy_table_2

,Los Angeles,Phoenix,San Diego,San Franciso,Seattle,Sum
Airline,,,,,,
ALASKA,15.2,6.8,6.5,15.4,56.2,100.0
AM WEST,10.8,75.2,5.9,5.0,3.1,100.0


This data frame shows the percentage of on time arrivals for each city by both the airlines. 
Which shows that ALASKA airline outperforms AM WEST for each city except for Phoenix. AM WEST has 75.2% on time arrivals for Phoenix while ALASKA has just 6.8%.


## 2: Secondly, I will analyze the overall on time performance of both airlines.

In [10]:
tidy_table3 = tidy_table.groupby('Airline')['delayed','on time'].sum().reset_index()
tidy_table3

,Airline,delayed,on time
0,ALASKA,501,3274
1,AM WEST,787,6438


In [11]:
tidy_table3.loc[:,'Total']= tidy_table3.sum(axis=1)
val_1 = tidy_table3.loc[0,'Total']
val_A = tidy_table3.loc[0,'on time']
val_A = val_A/val_1
val_A = val_A*100
val_2 = tidy_table3.loc[1,'Total']
val_B = tidy_table3.loc[1,'on time']
val_B = val_B/val_2
val_B = val_B*100
tidy_table3.loc[:,'Overall on_time performance'] = [val_A,val_B]
tidy_table3

,Airline,delayed,on time,Total,Overall on_time performance
0,ALASKA,501,3274,3775,86.728477
1,AM WEST,787,6438,7225,89.107266


__Overall best perfomance is of AM WEST airline, which is 89% on time.__

# Long to Wide Format:

If I have to convert the long format that I have created above to a wide format, I would choose either of the following two structures. I find these easier to understand as compared to the one already provided. 

In [42]:
index = ['City']
wide_1 = tidy_table.pivot_table(index=index,columns='Airline')
wide_1

delayed         on time        
Airline       ALASKA AM WEST  ALASKA AM WEST
City                                        
Los Angeles       62     117     497     694
Phoenix           12     415     221    4840
San Diego         20      65     212     383
San Franciso     102     129     503     320
Seattle          305      61    1841     201

In [43]:
wide_2 = tidy_table.set_index(['Airline','City'])
wide_2

delayed  on time
Airline City                          
ALASKA  Los Angeles        62      497
        Phoenix            12      221
        San Diego          20      212
        San Franciso      102      503
        Seattle           305     1841
AM WEST Los Angeles       117      694
        Phoenix           415     4840
        San Diego          65      383
        San Franciso      129      320
        Seattle            61      201